# HPXPy Distribution API Demo

This notebook demonstrates the distribution infrastructure, showing how the API is structured for future distributed array support.

## Key Concepts

- **Locality**: An HPX execution context (typically one per node), with its own memory and OS threads
- **Partition**: A contiguous chunk of array data residing on a single locality
- **Block Distribution**: Divides array into contiguous chunks across localities
- **Cyclic Distribution**: Round-robin element assignment across localities

In [ ]:
import time
import numpy as np
import hpxpy as hpx

hpx.init(num_threads=4)

## 1. Distribution Policies

In [ ]:
print("Available distribution policies:")
print(f"  hpx.distribution.none   = {hpx.distribution.none}")
print(f"  hpx.distribution.block  = {hpx.distribution.block}")
print(f"  hpx.distribution.cyclic = {hpx.distribution.cyclic}")
print(f"  hpx.distribution.local  = {hpx.distribution.local} (alias for none)")

## 2. Locality Introspection

In [ ]:
locality_id = hpx.distribution.get_locality_id()
num_localities = hpx.distribution.get_num_localities()

print(f"Current locality ID: {locality_id}")
print(f"Number of localities: {num_localities}")

if num_localities == 1:
    print("\nNote: Running in single-locality mode.")
    print("In a distributed HPX deployment, these values would")
    print("reflect the actual cluster topology.")

## 3. Distribution Policy Enum

In [ ]:
print("The DistributionPolicy enum can be used for type-safe policy selection:")
print(f"  DistributionPolicy.none  = {hpx.distribution.DistributionPolicy.none}")
print(f"  DistributionPolicy.block = {hpx.distribution.DistributionPolicy.block}")
print(f"  DistributionPolicy.cyclic = {hpx.distribution.DistributionPolicy.cyclic}")

print("\nPolicy comparison:")
print(f"  none == local: {hpx.distribution.none == hpx.distribution.local}")
print(f"  block == cyclic: {hpx.distribution.block == hpx.distribution.cyclic}")

## 4. Array Operations (Unchanged)

All existing array operations continue to work:

In [ ]:
# Create arrays
a = hpx.arange(1000000)
b = hpx.arange(1000000)

# Perform operations
start = time.perf_counter()
c = a + b
c = hpx.sqrt(c)
result = hpx.sum(c)
elapsed = time.perf_counter() - start

print(f"Created two arrays of 1M elements")
print(f"Computed: sum(sqrt(a + b))")
print(f"Result: {result:.2f}")
print(f"Time: {elapsed*1000:.2f} ms")

## 5. Future Distribution API (Preview)

In future phases, distributed arrays will support:

```python
# Create distributed array across localities
arr = hpx.zeros((10000000,), distribution='block')

# Query distribution info
arr.is_distributed      # True if multi-locality
arr.num_partitions      # Number of data partitions
arr.partition_sizes     # Size of each partition
arr.localities          # Which localities hold data

# Gather to single locality
local = arr.to_numpy()  # Collects all data

# Distributed operations
result = hpx.sum(arr)   # Parallel reduction across localities
```

## 6. HPX Distributed Computing Concepts

| Concept | Description |
|---------|-------------|
| **Locality** | An HPX execution context (typically one per node). Each locality has its own memory and OS threads. |
| **Partition** | A contiguous chunk of array data residing on a single locality. |
| **Block Distribution** | Divides array into contiguous chunks. Elements `[0..N/P)` on locality 0, `[N/P..2N/P)` on locality 1, etc. |
| **Cyclic Distribution** | Round-robin element assignment. Element `i` goes to locality `(i % P)`. |

In [ ]:
hpx.finalize()
print("Demo complete!")